# Лабораторная работа №2
## Задание:
**Сформировать отчет с информацией о 10 наиболее популярных языках программирования по итогам года за период с 2010 по 2020 годы.

Получившийся отчет сохранить в формате `Apache Parquet`

In [1]:
from pyspark.sql import SparkSession

import pandas as pd
import os

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.14.0 pyspark-shell'

In [3]:
spark = SparkSession.builder.appName("LAB2").master("local[*]").getOrCreate()
spark

Читаем `posts_sample.xml` и изучаем DataFrame

In [4]:
posts_sample = pd.read_xml("data\posts_sample.xml")

In [7]:
posts_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46006 entries, 0 to 46005
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Id                     46006 non-null  int64  
 1   PostTypeId             46006 non-null  int64  
 2   AcceptedAnswerId       9510 non-null   float64
 3   CreationDate           46006 non-null  object 
 4   Score                  46006 non-null  int64  
 5   ViewCount              18055 non-null  float64
 6   Body                   45988 non-null  object 
 7   OwnerUserId            45459 non-null  float64
 8   LastEditorUserId       18283 non-null  float64
 9   LastEditorDisplayName  306 non-null    object 
 10  LastEditDate           18461 non-null  object 
 11  LastActivityDate       46006 non-null  object 
 12  Title                  18055 non-null  object 
 13  Tags                   18055 non-null  object 
 14  AnswerCount            18055 non-null  float64
 15  Co

In [131]:
posts_sample.head(2)

,Id,PostTypeId,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastEditorUserId,LastEditorDisplayName,...,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,CommunityOwnedDate,ParentId,OwnerDisplayName,ClosedDate
0,4,1,7.0,2008-07-31T21:42:52.667,630,42817.0,<p>I want to use a track-bar to change a form'...,8.0,3641067.0,Rich B,...,2019-07-19T01:39:54.173,Convert Decimal to Double?,<c#><floating-point><type-conversion><double><...,13.0,2,48.0,2012-10-31T16:42:47.213,NaN,None,None
1,6,1,31.0,2008-07-31T22:08:08.620,281,18214.0,<p>I have an absolutely positioned <code>div</...,9.0,3641067.0,Rich B,...,2019-07-19T01:43:04.077,Percentage width child element in absolutely p...,<html><css><internet-explorer-7>,6.0,0,10.0,None,NaN,None,None


Оставим только интересующие нас столбцы в DataFrame

In [132]:
sorted_ps = posts_sample[['CreationDate', 'Tags']]

Мы видим, что в dataframe есть множество постов без тэгов о используемых ЯП. Они нам не нужны. Удалим эти строки.

In [133]:
sorted_ps = sorted_ps.dropna()

Читаем и изучаем `programming-languages.csv`. Так как в нем есть `header` сразу уберем его. 

In [134]:
languages_name = spark.read.format('csv').option('header', 'true').load("data\programming-languages.csv")

In [135]:
languages_name.take(5)

[Row(name='A# .NET', wikipedia_url='https://en.wikipedia.org/wiki/A_Sharp_(.NET)'),
 Row(name='A# (Axiom)', wikipedia_url='https://en.wikipedia.org/wiki/A_Sharp_(Axiom)'),
 Row(name='A-0 System', wikipedia_url='https://en.wikipedia.org/wiki/A-0_System'),
 Row(name='A+', wikipedia_url='https://en.wikipedia.org/wiki/A%2B_(programming_language)'),
 Row(name='A++', wikipedia_url='https://en.wikipedia.org/wiki/A%2B%2B')]

Нам нужны только имена языков программирования.

In [136]:
pllist = []
for a in languages_name.collect():
    pllist.append(str(a[0]))

In [137]:
print (len(pllist))
pllist[0:10]

700


['A# .NET',
 'A# (Axiom)',
 'A-0 System',
 'A+',
 'A++',
 'ABAP',
 'ABC',
 'ABC ALGOL',
 'ABSET',
 'ABSYS']

Так как в `posts_sample` языки изображены в виде нижних регистров посреди знаков `< >`, то приведем наш список к такому же виду.

In [126]:
for c in range (len(pllist)):
    pllist[c] = f'<{pllist[c].lower()}>'

In [158]:
years = []
for i in range (2010, 2020):
    years.append(i)

In [159]:
result = []
for year in years:
    year_res = []
    for x in range(len(pllist)):
        year_res.append([])
    for p in range(len(sorted_ps)):
        date = sorted_ps.iloc[p]['CreationDate']
        if str(year) in date:
            for c in range(len(pllist)):
                tag = sorted_ps.iloc[p]['Tags']
                if pllist[c] in tag:
                    year_res[c].append(1)
    result.append(year_res)

In [162]:
for lang in result[0]:
    if sum(lang) != 0:
        print(sum(lang))

1
2
103
1
20


In [144]:
result

[]